# BDP - SQL Workshop

**Recitation notebook**

---

Create an account and download the train.csv file from kaggle's titanic competition

https://www.kaggle.com/c/titanic/data

Move the csv file to the same path as the notebook

---

**The titanic🛳️ dataset is one of the most famous Machine Learning datasets out there.**

**Dataset columns:** <br>
passengerid - *Unique identifier for the passenger* <br>
pclass - *Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)* <br>
name - *Name* <br>
sex - *Sex* <br>
age - *Age* <br>
sibsp - *Number of Siblings / Spouses Aboard* <br>
parch - *Number of Parents / Children Aboard* <br>
ticket - *Ticket Number* <br>
fare - *Passenger Fare* <br>
cabin - *Cabin* <br>
embarked - *Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)* <br>
survived - *Survival (0 = No; 1 = Yes)* <br>

---

**SETUP**

For the following exercises we will use the following libraries: <br>
- pandas (tabular data) https://pandas.pydata.org/docs/
- sqlite database engine https://www.sqlite.org/index.html <br>

In [4]:
import pandas as pd
import sqlite3

In [30]:
df = pd.read_csv('train.csv')

df.columns = df.columns.str.lower()

# Split the data into features and labels (we will store them in a seperate tables)
X = df.drop('survived', axis=1)
y = df[['passengerid', 'survived']]

---

Connect to database

In [31]:
con = sqlite3.connect('bdp.db')

Write to database (with pandas)

In [32]:
X.to_sql('titanic_features', con=con, index=False, if_exists='replace')
y.to_sql('titanic_labels', con=con, index=False, if_exists='replace')

Make sure you can query over the database

In [8]:
query = """

SELECT * FROM titanic_features

"""

pd.read_sql_query(query, con=con)

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,None,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


---

## GROUP BY & HAVING

#### GROUP BY 
- groups rows with same values into summary rows

#### HAVING
- filters the results of GROUP BY 

#### HAVING vs. WHERE
- WHERE filters individual rows, HAVING filters GROUP BY summary rows, like WHERE for GROUP BY

In [38]:
query = """

SELECT embarked
     , COUNT(*) AS passengers
FROM titanic_features
GROUP BY embarked
-- HAVING embarked NOT NULL
HAVING COUNT(*) > 10

"""

pd.read_sql_query(query, con=con)

,embarked,passengers
0,C,168
1,Q,77
2,S,644


## COUNT(*), COUNT(column), COUNT(DISTINCT column), COUNT(CASE)

#### COUNT (*) 
- counts all rows in the table column

#### COUNT(column) 
- counts all NOT NULL rows in the column

#### COUNT (DISTINCT column) 
- counts all unique NOT NULL rows in the column (no duplicates)

In [10]:
query = """

SELECT embarked
     , COUNT(*) AS passengers
     , COUNT(age) AS passengers_with_age
     , COUNT(DISTINCT age) AS passengers_with_different_age
FROM titanic_features
WHERE embarked NOT NULL
GROUP BY 1

"""

pd.read_sql_query(query, con=con)

,embarked,passengers,passengers_with_age,passengers_with_different_age
0,C,168,130,58
1,Q,77,28,24
2,S,644,554,80


#### COUNT(CASE)
- CASE WHEN is like if / then and COUNT(CASE) tallies the TRUEs

#### COUNT(CASE)

In [11]:
query = """

SELECT embarked
     , COUNT(*) AS passengers
     , COUNT(CASE WHEN LOWER(name) LIKE '%mr.%' THEN 1 END) AS misters
FROM titanic_features
WHERE embarked NOT NULL
GROUP BY 1

"""

pd.read_sql_query(query, con=con)

,embarked,passengers,misters
0,C,168,85
1,Q,77,35
2,S,644,397


In [12]:
query = """

SELECT embarked
     , COUNT(*) AS passengers
     , COUNT(CASE WHEN LOWER(name) LIKE '%mr.%' THEN 1 END) AS misters
FROM titanic_features
WHERE embarked NOT NULL
GROUP BY 1
HAVING COUNT(CASE WHEN LOWER(name) LIKE '%mr.%' THEN 1 END) > 50

"""

pd.read_sql_query(query, con=con)

,embarked,passengers,misters
0,C,168,85
1,S,644,397


#### Don't forget to write your queries properly! 
- It's not just about the result... need to consider efficiency, maintenance and readability

#### An example of what NOT to do...

In [13]:
query = """

select embarked, count(*) passengers, count(CASE WHEN lower(name) LIKE '%mr.%' THEN 1 END) AS misters from titanic_features
where embarked not null group by embarked
having count(case when lower(name)
like '%mr.%' then 1 end) > 50

"""

pd.read_sql_query(query, con=con)

,embarked,passengers,misters
0,C,168,85
1,S,644,397


Its not all about the result! take into consideration efficiency and maintenance.

---

## WINDOW FUNCTIONS

- WINDOW functions vs. aggregate functions comparison

- SUM() will result in one single value/row in the output

- SUM() OVER() will output the sum in every row 

- PARTITION BY is similar to a GROUP BY inside of a SELECT WINDOW

In [14]:
query = """

SELECT passengerid
     , name
     , age
     , pclass
     , cabin
     , fare
     , SUM(fare) OVER() AS total_fare
FROM titanic_features
WHERE cabin NOT NULL
    
"""

pd.read_sql_query(query, con=con)

,passengerid,name,age,pclass,cabin,fare,total_fare
0,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,C85,71.2833,15532.8668
1,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,C123,53.1000,15532.8668
2,7,"McCarthy, Mr. Timothy J",54.0,1,E46,51.8625,15532.8668
3,11,"Sandstrom, Miss. Marguerite Rut",4.0,3,G6,16.7000,15532.8668
4,12,"Bonnell, Miss. Elizabeth",58.0,1,C103,26.5500,15532.8668
...,...,...,...,...,...,...,...
199,872,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",47.0,1,D35,52.5542,15532.8668
200,873,"Carlsson, Mr. Frans Olof",33.0,1,B51 B53 B55,5.0000,15532.8668
201,880,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",56.0,1,C50,83.1583,15532.8668
202,888,"Graham, Miss. Margaret Edith",19.0,1,B42,30.0000,15532.8668


In [15]:
query = """

SELECT passengerid
     , name
     , age
     , pclass
     , cabin
     , fare
     , SUM(fare) OVER() AS total_fare
     , SUM(fare) OVER(PARTITION BY cabin) AS total_fare_by_cabin
FROM titanic_features
WHERE cabin NOT NULL
    
"""

pd.read_sql_query(query, con=con)

,passengerid,name,age,pclass,cabin,fare,total_fare,total_fare_by_cabin
0,584,"Ross, Mr. John Hugo",36.0,1,A10,40.1250,15532.8668,40.1250
1,476,"Clifford, Mr. George Quincy",NaN,1,A14,52.0000,15532.8668,52.0000
2,557,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",48.0,1,A16,39.6000,15532.8668,39.6000
3,285,"Smith, Mr. Richard William",NaN,1,A19,26.0000,15532.8668,26.0000
4,600,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",49.0,1,A20,56.9292,15532.8668,56.9292
...,...,...,...,...,...,...,...,...
199,11,"Sandstrom, Miss. Marguerite Rut",4.0,3,G6,16.7000,15532.8668,54.3250
200,206,"Strom, Miss. Telma Matilda",2.0,3,G6,10.4625,15532.8668,54.3250
201,252,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",29.0,3,G6,10.4625,15532.8668,54.3250
202,395,"Sandstrom, Mrs. Hjalmar (Agnes Charlotta Bengt...",24.0,3,G6,16.7000,15532.8668,54.3250


In [16]:
query = """

SELECT passengerid
     , name
     , age
     , pclass
     , cabin
     , fare
     , SUM(fare) OVER(PARTITION BY cabin ORDER BY age DESC) AS cumulative_fare
     , COUNT(passengerid) OVER(PARTITION BY cabin) AS cabin_passengers
FROM titanic_features
WHERE cabin NOT NULL
    
"""

pd.read_sql_query(query, con=con)

,passengerid,name,age,pclass,cabin,fare,cumulative_fare,cabin_passengers
0,584,"Ross, Mr. John Hugo",36.0,1,A10,40.1250,40.1250,1
1,476,"Clifford, Mr. George Quincy",NaN,1,A14,52.0000,52.0000,1
2,557,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",48.0,1,A16,39.6000,39.6000,1
3,285,"Smith, Mr. Richard William",NaN,1,A19,26.0000,26.0000,1
4,600,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",49.0,1,A20,56.9292,56.9292,1
...,...,...,...,...,...,...,...,...
199,252,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",29.0,3,G6,10.4625,10.4625,4
200,395,"Sandstrom, Mrs. Hjalmar (Agnes Charlotta Bengt...",24.0,3,G6,16.7000,27.1625,4
201,11,"Sandstrom, Miss. Marguerite Rut",4.0,3,G6,16.7000,43.8625,4
202,206,"Strom, Miss. Telma Matilda",2.0,3,G6,10.4625,54.3250,4


---

## JOINS

#### Types of JOINs
- INNER JOIN
- LEFT JOIN / RIGHT JOIN
- SELF JOIN... ancestor of WINDOW functions

#### SELF JOIN

In [17]:
query = """

SELECT t1.passengerid
     , t1.name
     , t1.age
     , t1.pclass
     , t1.cabin
     , t1.fare
     , SUM(t2.fare) AS cumulative_fare
FROM titanic_features AS t1
JOIN titanic_features AS t2
  ON t1.cabin = t2.cabin
    AND t1.age <= t2.age
GROUP BY 1,2,3,4,5,6
ORDER BY t1.cabin, t1.age DESC

"""

pd.read_sql_query(query, con=con)

,passengerid,name,age,pclass,cabin,fare,cumulative_fare
0,584,"Ross, Mr. John Hugo",36.0,1,A10,40.1250,40.1250
1,557,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",48.0,1,A16,39.6000,39.6000
2,600,"Duff Gordon, Sir. Cosmo Edmund (""Mr Morgan"")",49.0,1,A20,56.9292,56.9292
3,631,"Barkworth, Mr. Algernon Henry Wilson",80.0,1,A23,30.0000,30.0000
4,868,"Roebling, Mr. Washington Augustus II",31.0,1,A24,50.4958,50.4958
...,...,...,...,...,...,...,...
180,252,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",29.0,3,G6,10.4625,10.4625
181,395,"Sandstrom, Mrs. Hjalmar (Agnes Charlotta Bengt...",24.0,3,G6,16.7000,27.1625
182,11,"Sandstrom, Miss. Marguerite Rut",4.0,3,G6,16.7000,43.8625
183,206,"Strom, Miss. Telma Matilda",2.0,3,G6,10.4625,54.3250


#### INNER JOIN

In [18]:
query = """

SELECT *
FROM titanic_features AS t1
  JOIN titanic_labels AS t2
      ON t1.passengerid = t2.passengerid

"""

pd.read_sql_query(query, con=con)

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,passengerid,survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,3,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,4,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S,887,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,888,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,None,S,889,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,890,1


In [19]:
query = """

SELECT *
FROM titanic_features AS t1, titanic_labels AS t2
WHERE t1.passengerid = t2.passengerid

"""

pd.read_sql_query(query, con=con)

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,passengerid,survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,3,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,4,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S,887,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,888,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,None,S,889,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,890,1


In [20]:
query = """

SELECT *
FROM titanic_features
  JOIN titanic_labels
      USING(passengerid)

"""

pd.read_sql_query(query, con=con)

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,None,S,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,None,S,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


---

## CTEs

- with CTEs there is no temp table created 
- makes queries more readable and more efficient to maintain

**Machine Learning - Classification**

In [21]:
query = """

SELECT CASE WHEN sex = 'male' THEN 0 ELSE 1 END AS y_pred
     , survived AS y_true
FROM titanic_features
  JOIN titanic_labels
    USING(passengerid)

"""

pd.read_sql_query(query, con=con)

,y_pred,y_true
0,0,0
1,1,1
2,1,1
3,1,1
4,0,0
...,...,...
886,0,0
887,1,1
888,1,0
889,0,1


In [22]:
query = """

WITH raw_data AS (
  SELECT CASE WHEN sex = 'male' THEN 0 ELSE 1 END AS y_pred
       , survived AS y_true
  FROM titanic_features
    JOIN titanic_labels
      USING(passengerid)
  )

SELECT AVG(y_true = y_pred) AS accuracy
FROM raw_data
    
"""

pd.read_sql_query(query, con=con)

,accuracy
0,0.786756


#### An example of what NOT to do (nested queries)...
   - especially for more than one subquery

In [23]:
query = """

SELECT AVG(y_true = y_pred) AS accuracy
FROM (SELECT CASE WHEN sex = 'male' THEN 0 ELSE 1 END AS y_pred
           , survived AS y_true
      FROM titanic_features
        JOIN titanic_labels
          USING(passengerid)
      )
    
"""

pd.read_sql_query(query, con=con)

,accuracy
0,0.786756


---

## Dynamic Queries
- Dynamic queries enable us to have changeable variables inside our queries, by putting the query inside of a python function, as we'll demonstrate below. 


- The query stays the same, but *dynamic params* are inserted to the base template of the query string.

In [24]:
def search_family(family_name):
    
    family_name = family_name.lower()
    
    query = f"""

    SELECT *
    FROM titanic_features
    WHERE LOWER(name) LIKE '%{family_name},%'

    """

    return pd.read_sql_query(query, con=con)

- Note the usage of LOWER(name) and the python preprocess step to avoid issues with capitalized letters in names.

In [25]:
search_family(family_name='Fortune')

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,28,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0,C23 C25 C27,S
1,89,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0,C23 C25 C27,S
2,342,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0,C23 C25 C27,S
3,439,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0,C23 C25 C27,S


In [26]:
search_family(family_name='sage')

,passengerid,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,160,3,"Sage, Master. Thomas Henry",male,None,8,2,CA. 2343,69.55,None,S
1,181,3,"Sage, Miss. Constance Gladys",female,None,8,2,CA. 2343,69.55,None,S
2,202,3,"Sage, Mr. Frederick",male,None,8,2,CA. 2343,69.55,None,S
3,325,3,"Sage, Mr. George John Jr",male,None,8,2,CA. 2343,69.55,None,S
4,793,3,"Sage, Miss. Stella Anna",female,None,8,2,CA. 2343,69.55,None,S
5,847,3,"Sage, Mr. Douglas Bullen",male,None,8,2,CA. 2343,69.55,None,S
6,864,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,None,8,2,CA. 2343,69.55,None,S


---

Terminate the connection to the database

In [27]:
con.close()

---